# Iteration 3: organise parameters

## Plan English Summary

Before we add in results collection the organisation of the model is modified so that parameters are separated from model logic.  Given the "hidden code" issue in iteration 2 the input included a command to show all generator functions.

The same tests can be applied here as iteration 1, 2, and 3. These should now be easier as parameters should not be hardcoded and are instead accessible via the Experiment class.




## Notebook imports

In [7]:
from IPython.display import Markdown, display

import numpy as np
import math

## Show query 

In [10]:
display(Markdown("inputs/iteration_3.md"))

## Main command

Modify the code so that all model parameters are stored within a class called Experiment that is separate from the model's logic.  

The list of parameters include:

1. the means of the arrival distributions
2. the treatment time parameters
3. The number of critical care beds available
4. The duration of intensive cleaning following a patient discharge.
5. simulation run length

The parameters should be passed to the Experiment class constructor method when it is created. please provide default values

Show all of the model code: this includes all patient generator functions in the CCModel class.

## Date and time of input to LLM

> 1500 29th November 2023 

## Observations on the LLM Output

* The `Experiment` class has been created.  All parameters are passed in the constructor method.  This uses a tuple.
* The mean elective treatment time is included in `stay_distributions`, but there is also an `elective_treatment_mean` variable.  It is this variable that is used by `elective_surgery_process` to sample from the exponential distribution.
* There is a bug in the code. The `run` method uses the module level scope variable `env` instead of the instance variable `self.env` that `CCUModel` has been passed.  This was picked up by the tester.  The LLM was asked to fix the bug.

## LLM Output 1:

Here's the modified code with all model parameters stored within a separate class called `Experiment`:

In this modified code, all model parameters are stored within a separate class called `Experiment`. The `Experiment` class is responsible for storing the means of the arrival distributions, the unplanned patient treatment time mean and standard deviations, the elective patients mean treatment time, the number of critical care beds available, the duration of intensive cleaning following a patient discharge, and the simulation run length. These parameters are passed to the `Experiment` class constructor method when it is created, with default values provided.

The `CCUModel` class includes all patient generator functions, and it receives an instance of the `Experiment` class to access the model parameters. The simulation environment is created using `simpy.Environment()`, and an instance of the `CCUModel` is created to run the simulation.

This code is based on the requirements specified and uses SimPy 4.1.1 and Python 3.10.4, as requested. For more details on SimPy and Python, please refer to the official documentation and release notes.

Citations:
[1] https://towardsdatascience.com/object-oriented-discrete-event-simulation-with-simpy-53ad82f5f6e2

[2] https://annalsofintensivecare.springeropen.com/articles/10.1186/2110-5820-2-18

[3] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC7491890/

[4] https://biomedical-engineering-online.biomedcentral.com/articles/10.1186/s12938-018-0455-y

[5] https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8189840/

In [11]:
import simpy
import numpy as np

class Experiment:
    def __init__(self, 
                 interarrival_means=[22.72, 26.0, 37.0, 47.2, 575.0, 17.91], 
                 stay_distributions=[(128.79, 267.51), (177.89, 276.54), (140.15, 218.02), (212.86, 457.67), (87.53, 108.67), 57.34], 
                 elective_treatment_mean=57.34, 
                 num_critical_care_beds=24, 
                 intensive_cleaning_duration=5, 
                 run_length=12*30*24):
        self.interarrival_means = interarrival_means
        self.stay_distributions = stay_distributions
        self.elective_treatment_mean = elective_treatment_mean
        self.num_critical_care_beds = num_critical_care_beds
        self.intensive_cleaning_duration = intensive_cleaning_duration
        self.run_length = run_length

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def run(self):
        env.process(self.patient_arrival_AE())
        env.process(self.patient_arrival_wards())
        env.process(self.patient_arrival_surgery())
        env.process(self.patient_arrival_other_hospitals())
        env.process(self.patient_arrival_X_ray())
        env.process(self.patient_arrival_elective_surgery())
        env.run(until=self.experiment.run_length)

# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()

Patient 1 arrived from the Wards at time 14.258788922018885
Patient 1 admitted to critical care bed at time 14.258788922018885
Patient 2 arrived from Accident and Emergency at time 14.845301732799431
Patient 2 admitted to critical care bed at time 14.845301732799431
Elective surgery patient 3 arrived at time 18.978767338441408
Elective surgery patient 3 admitted to critical care bed at time 18.978767338441408
Patient 4 arrived from other hospitals at time 33.59875243243078
Patient 4 admitted to critical care bed at time 33.59875243243078
Patient 4 discharged from critical care bed at time 34.74495611028114
Elective surgery patient 5 arrived at time 36.93849674414986
Elective surgery patient 5 admitted to critical care bed at time 36.93849674414986
Patient 6 arrived from Emergency surgery at time 38.762404948736666
Patient 6 admitted to critical care bed at time 38.762404948736666
Patient 7 arrived from Emergency surgery at time 38.982612904103306
Patient 7 admitted to critical care bed

## ADDITIONAL COMMAND/INPUT

The run method of the CCUModel class contains a bug.  The method should refer to self.env instead of the variable env

## LLM Output 2

Here's the complete code for the CCUModel class with the run method corrected to refer to self.env:

In this corrected code, the run method correctly refers to self.env instead of the variable env. This ensures that the simulation environment is accessed through the instance of the CCUModel class.

Citations:
[1] https://github.com/pyinvoke/invoke/issues/259

[2] https://classroomcheckup.org

[3] https://stackoverflow.com/questions/68493263/create-method-with-class-self-variables-outside-of-a-python-class-and-then-add-i

[4] https://github.com/spack/spack/discussions/13926

[5] https://www.cms.gov/medicare-coverage-database/view/article.aspx?articleId=52985

In [28]:
import simpy
import numpy as np

class CCUModel:
    def __init__(self, env, experiment):
        self.env = env
        self.experiment = experiment
        self.patient_count = 0
        self.critical_care_beds = simpy.Resource(env, capacity=experiment.num_critical_care_beds)

    def patient_arrival_AE(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[0]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Accident and Emergency at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[0]))

    def patient_arrival_wards(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[1]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the Wards at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[1]))

    def patient_arrival_surgery(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[2]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from Emergency surgery at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[2]))

    def patient_arrival_other_hospitals(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[3]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from other hospitals at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[3]))

    def patient_arrival_X_ray(self):
        while True:
            yield self.env.timeout(np.random.exponential(self.experiment.interarrival_means[4]))
            self.patient_count += 1
            print(f"Patient {self.patient_count} arrived from the X-Ray department at time {self.env.now}")
            self.env.process(self.unplanned_admission(self.experiment.stay_distributions[4]))

    def patient_arrival_elective_surgery(self):
        while True:
            yield self.env.timeout(np.random.normal(self.experiment.interarrival_means[5], 3.16))
            self.patient_count += 1
            print(f"Elective surgery patient {self.patient_count} arrived at time {self.env.now}")
            if len(self.critical_care_beds.users) == self.critical_care_beds.capacity:
                print(f"Elective surgery for patient {self.patient_count} cancelled due to no available critical care beds at time {self.env.now}")
            else:
                self.env.process(self.elective_surgery_process(self.experiment.elective_treatment_mean))

    def unplanned_admission(self, stay_distribution):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.lognormal(np.log(stay_distribution[0]) - 0.5 * np.log(1 + (stay_distribution[1] / stay_distribution[0])**2),
                                                 np.sqrt(np.log(1 + (stay_distribution[1] / stay_distribution[0])**2)))
            yield self.env.timeout(treatment_time)
            print(f"Patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def elective_surgery_process(self, treatment_mean):
        with self.critical_care_beds.request() as req:
            yield req
            print(f"Elective surgery patient {self.patient_count} admitted to critical care bed at time {self.env.now}")
            treatment_time = np.random.exponential(treatment_mean)
            yield self.env.timeout(treatment_time)
            print(f"Elective surgery patient {self.patient_count} discharged from critical care bed at time {self.env.now}")
            yield self.env.timeout(self.experiment.intensive_cleaning_duration)
            print(f"Critical care bed is available for next patient at time {self.env.now}")

    def run(self):
        self.env.process(self.patient_arrival_AE())
        self.env.process(self.patient_arrival_wards())
        self.env.process(self.patient_arrival_surgery())
        self.env.process(self.patient_arrival_other_hospitals())
        self.env.process(self.patient_arrival_X_ray())
        self.env.process(self.patient_arrival_elective_surgery())
        self.env.run(until=self.experiment.run_length)


In [29]:
# Create a SimPy environment
env = simpy.Environment()

# Create an instance of the Experiment class and the CCU model, then run the simulation
experiment = Experiment()
ccu_model = CCUModel(env, experiment)
ccu_model.run()

Patient 1 arrived from Accident and Emergency at time 6.6279700100840815
Patient 1 admitted to critical care bed at time 6.6279700100840815
Patient 2 arrived from Emergency surgery at time 13.103435130559095
Patient 2 admitted to critical care bed at time 13.103435130559095
Elective surgery patient 3 arrived at time 21.24345713356632
Elective surgery patient 3 admitted to critical care bed at time 21.24345713356632
Patient 4 arrived from Emergency surgery at time 23.346255778846064
Patient 4 admitted to critical care bed at time 23.346255778846064
Patient 5 arrived from the Wards at time 31.838570589615543
Patient 5 admitted to critical care bed at time 31.838570589615543
Patient 6 arrived from Emergency surgery at time 31.891308282952785
Patient 6 admitted to critical care bed at time 31.891308282952785
Patient 6 discharged from critical care bed at time 34.42093287984717
Elective surgery patient 7 arrived at time 38.650004891390886
Elective surgery patient 7 admitted to critical care

## Modifications to LLM code to test for correct model outputs.

Here we do a number of regression tests to check that previous functionality is not affected by the `Experiment` class.

From iteration 0

* **Extreme value test 1**: Wards, Em Surgery, other hospitals, x-ray have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Accident and Emergency."
* **Extreme value test 2**: All unplanned patient types have have their inter-arrival time is set to $M$ a very large number
    * Expected result: The only type of patient to arrive to the model is "Elective."
    * Actual result **(PASS)**: The only type of patient to arrive to the model is "Elective."
* **Different run length**: The run length of the model is set to 10 days.
    * Expected result: The model should runs no longer than 240 simulated time periods
    * Actual result **(PASS)**: The model runs no longer than 240 simulated time periods
* **Extreme value test 3**: `critical_care_beds` = 1
    * Expected result: queues form after first arrival.
    * Actual result **(PASS)**: queues form after first arrival. 
* **Extreme value test 4**: means of stay_distributions set to $M$ a very large number.
    * Expected result: after 24 arrivals queues form and no patients are admitted.
    * Actual result **(PASS)**: after 24 arrivals queues form and no patients are admitted.


### Test 1: Extreme value test 1

PASS

In [32]:
def run_test(experiment):
    # Create a SimPy environment
    env = simpy.Environment()
    ccu_model = CCUModel(env, experiment)
    ccu_model.run()
    return ccu_model

In [33]:
def extreme_value_test_1(extreme_value=M):
    '''
    Extreme value test 1: 
    
    Wards, Em Surgery, other hospitals, x-ray, electives have their inter-arrival time is
    set to $M$ a very large number
    Expected result: The only type of patient to arrive to the model is "Accident and Emergency."
    '''
    mod_interarrival_means = [22.72] + [extreme_value] * 5 
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [34]:
M = 10_000_000
extreme_value_test_1()

Patient 1 arrived from Accident and Emergency at time 1.7466846870478734
Patient 1 admitted to critical care bed at time 1.7466846870478734
Patient 2 arrived from Accident and Emergency at time 19.731184760543673
Patient 2 admitted to critical care bed at time 19.731184760543673
Patient 2 discharged from critical care bed at time 26.871237451816175
Critical care bed is available for next patient at time 31.871237451816175
Patient 3 arrived from Accident and Emergency at time 32.18186611136238
Patient 3 admitted to critical care bed at time 32.18186611136238
Patient 3 discharged from critical care bed at time 39.47863637680151
Critical care bed is available for next patient at time 44.47863637680151
Patient 3 discharged from critical care bed at time 60.08275033190679
Patient 4 arrived from Accident and Emergency at time 62.9955875341044
Patient 4 admitted to critical care bed at time 62.9955875341044
Critical care bed is available for next patient at time 65.0827503319068
Patient 5 arr

### Test 2: Extreme value test 2

PASS

In [35]:
def extreme_value_test_2(extreme_value=M):
    '''
    Extreme value test 2: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    mod_interarrival_means=[extreme_value] * 5 + [17.91]
    experiment = Experiment(interarrival_means=mod_interarrival_means)
    run_test(experiment)

In [36]:
M = 10_000_000
extreme_value_test_2()

Elective surgery patient 1 arrived at time 19.988569495620926
Elective surgery patient 1 admitted to critical care bed at time 19.988569495620926
Elective surgery patient 2 arrived at time 38.462301919256504
Elective surgery patient 2 admitted to critical care bed at time 38.462301919256504
Elective surgery patient 3 arrived at time 58.521316589316456
Elective surgery patient 3 admitted to critical care bed at time 58.521316589316456
Elective surgery patient 3 discharged from critical care bed at time 69.4352941084738
Elective surgery patient 4 arrived at time 73.10702837170254
Elective surgery patient 4 admitted to critical care bed at time 73.10702837170254
Critical care bed is available for next patient at time 74.4352941084738
Elective surgery patient 5 arrived at time 94.72572811459602
Elective surgery patient 5 admitted to critical care bed at time 94.72572811459602
Elective surgery patient 5 discharged from critical care bed at time 98.1133944168323
Critical care bed is availabl

### Test 3: Different run length

PASS

In [40]:
def test_run_length(new_run_length):
    '''
    Different run length

    Expected result Run length should be no longer than `new_run_length`
    '''
    experiment = Experiment(run_length=new_run_length)
    model = run_test(experiment)
    return model.env.now

In [41]:
test_run_length(10)

Patient 1 arrived from Accident and Emergency at time 8.083738237744715
Patient 1 admitted to critical care bed at time 8.083738237744715
Patient 2 arrived from other hospitals at time 8.563555522158937
Patient 2 admitted to critical care bed at time 8.563555522158937


10

### Test 4: Extreme value test 3

PASS.

In [42]:
def extreme_value_test_3(critical_care_beds=1):
    '''
    Extreme value test 3: 
    
    All unplanned admissions have their inter-arrival time is
    set to $M$ a very large number
    
    Expected result: The only type of patient to arrive to the model is "Elective"
    '''
    experiment = Experiment(num_critical_care_beds=critical_care_beds)
    run_test(experiment)

In [43]:
extreme_value_test_3()

Patient 1 arrived from the Wards at time 4.631035874898535
Patient 1 admitted to critical care bed at time 4.631035874898535
Patient 2 arrived from Accident and Emergency at time 7.286732540219057
Elective surgery patient 3 arrived at time 17.81726447677158
Elective surgery for patient 3 cancelled due to no available critical care beds at time 17.81726447677158
Patient 4 arrived from Emergency surgery at time 29.763232759699907
Patient 5 arrived from the Wards at time 30.280782316437893
Elective surgery patient 6 arrived at time 32.79598566387254
Elective surgery for patient 6 cancelled due to no available critical care beds at time 32.79598566387254
Patient 7 arrived from the Wards at time 41.75629994795454
Patient 8 arrived from other hospitals at time 41.8025101125904
Patient 8 discharged from critical care bed at time 42.05034477729325
Patient 9 arrived from Accident and Emergency at time 45.38362864821134
Critical care bed is available for next patient at time 47.05034477729325
Pa

### Test 5: Extreme value test 4

PASS

In [44]:
def extreme_value_test_4(extreme_value=M):
    '''
    Extreme value test 4: 
    
    means of stay_distributions set to $M$ a very large number.
    Expected result: after 24 arrivals queues form and no patients are admitted.
    '''
    mod_stay_distributions=[(M, 267.51), 
                            (M, 276.54), 
                            (M, 218.02), 
                            (M, 457.67), 
                            (M, 108.67), 
                            57.34] 
    experiment = Experiment(stay_distributions=mod_stay_distributions, 
                            elective_treatment_mean=M)
    run_test(experiment)

In [45]:
extreme_value_test_4()

Patient 1 arrived from the Wards at time 0.6983194696418966
Patient 1 admitted to critical care bed at time 0.6983194696418966
Patient 2 arrived from Emergency surgery at time 7.797899808640909
Patient 2 admitted to critical care bed at time 7.797899808640909
Patient 3 arrived from Accident and Emergency at time 14.80287290991444
Patient 3 admitted to critical care bed at time 14.80287290991444
Patient 4 arrived from other hospitals at time 20.357590218570515
Patient 4 admitted to critical care bed at time 20.357590218570515
Patient 5 arrived from the Wards at time 20.38481691679999
Patient 5 admitted to critical care bed at time 20.38481691679999
Elective surgery patient 6 arrived at time 25.02465009808305
Elective surgery patient 6 admitted to critical care bed at time 25.02465009808305
Patient 7 arrived from Accident and Emergency at time 29.82504567462799
Patient 7 admitted to critical care bed at time 29.82504567462799
Patient 8 arrived from Accident and Emergency at time 32.81330